In [ ]:
!pip install nltk
!pip install Sastrawi
!pip install emoji==1.7
!pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 6.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 9.8 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=67d6eb2c7be495deedd82e73186f2b4e16511cab6e72acd7a94df03d04f94e4f
  Stored in directory: /root/.cache/pip/wheels/5e/8c/80/c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import re
import emoji
import demoji
import nltk
from emoji import emojize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#from google.colab import files
#uploaded = files.upload()
df1= pd.read_csv('/content/Indonesian Sentiment Twitter Dataset Labeled.csv')
#df1 = pd.read_csv('/content/drive/MyDrive/college/TTTTTTAAAAAAAA/dataset TA/dataset 7636.csv')
#df1 = pd.read_csv('/content/drive/MyDrive/college/TTTTTTAAAAAAAA/dataset TA/data booster -1jun.csv')
slang_dict = pd.read_csv('/content/drive/MyDrive/college/TTTTTTAAAAAAAA/dataset TA/colloquial-indonesian-lexicon.csv', usecols=['slang','formal'])
slang_dict = dict(zip(slang_dict['slang'], slang_dict['formal']))
list_stopwords= set(list_stopwords)

In [ ]:
df1.shape

(7636, 3)

In [ ]:
def casefold(df):
  lower_case = df.text.lower()  
  return lower_case

def rt_mentions(df):
  rt = re.findall('(rt @\w+:)', df.casefold)
  mentions = re.findall('(@\w+)', df.casefold)
  all_mentions = rt + mentions
  return all_mentions

def no_mentions(df):
  rt = re.sub('rt @\w+: ','', df.casefold)
  mentions = re.sub('@\w+','', rt)
  return mentions

def hashtags(df):
  hashtag = re.findall('(#\w+)', df.extract_mentions)
  return hashtag

def no_hashtags(df):
  hashtag = re.sub('(#\w+)','', df.extract_mentions)
  return hashtag

def demoji(df):
  return ''.join(c for c in df.extract_hashtags if c in emoji.UNICODE_EMOJI['en'])

def cleaning(df):
    c1 = re.sub('http\S+', '', df.extract_hashtags)       #URLs
    c2 = re.sub(r'\n+', ' ', c1)                          #Whitespaces
    c3 = re.sub("'", '', c2)                              #'
    c4 = re.sub('[^\w\s]', ' ', c3)                       #Punctuations
    c5 = re.sub('\d', ' ', c4)                            #Digits
    c6 = re.sub('a{2,}', 'a',c5)                          #aaaaaaa
    c7 = re.sub('\s+', ' ', c6)                           #Overspacings
    c8 = re.sub('(^\s)|(\s$)', '', c7)                    #Space on the start/end of string
    return ' '.join([w for w in c8.split() if len(w)>2])  

def normalize(df):
  tokens = re.split(r'\s', df.cleaned)
  return ' '.join(slang_dict.get(word, word) for word in tokens)

def stem(df):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return stemmer.stem(df.normalized)

def stopword(df):
  factory = StopWordRemoverFactory()
  remover = factory.create_stop_word_remover()
  removed = remover.remove(df.stemmed)
  removed = re.split(r'\s', removed)
  return [i for i in removed if i not in list_stopwords]

def cleantext(df):
  return ' '.join(i for i in df.tokens)
  

df1['casefold'] = df1.apply(casefold, axis=1)
  #df1['rts_and_mentions'] = df1.apply(rt_mentions, axis=1)
df1['extract_mentions'] = df1.apply(no_mentions, axis=1)
  #df1['hashtags'] = df1.apply(hashtags, axis=1)
df1['extract_hashtags'] = df1.apply(no_hashtags, axis=1)
df1['emoji'] = df1.apply(demoji, axis=1)
df1['cleaned'] = df1.apply(cleaning, axis=1)
  #df1['transfer_emoji'] = df1.apply(no_emoji, axis=1)
  #df1['extract_mentions'].apply(lambda x: emoji.demojize(x))
df1['normalized'] = df1.apply(normalize, axis=1)
df1['stemmed'] = df1.apply(stem, axis=1)
df1['tokens'] = df1.apply(stopword, axis=1)
df1['cleantext'] = df1.apply(cleantext, axis=1)


df1 = df1.drop_duplicates(subset='cleantext', keep='first')
df1.head()

,label,text,casefold,extract_mentions,extract_hashtags,emoji,cleaned,normalized,stemmed,tokens,cleantext
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,,lagu bosan apa yang aku save huhuhuhuhuhuhuhuh...,lagu bosan apa yang aku save huhuhuhuhuhuhuhuh...,lagu bosan apa yang aku save huhuhuhuhuhuhuhuh...,"[lagu, bosan, save, huhuhuhuhuhuhuhuhuhuuuuuuu...",lagu bosan save huhuhuhuhuhuhuhuhuhuuuuuuuuuuuuuu
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjut saja diam ini hingga kau dan aku e...,"[diam, kau, erti, saman, libat, hati]",diam kau erti saman libat hati
2,1,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,,doa rezeki tak putus inna larizquna lahu min f...,doa rezeki tak putus inna larizquna lahu min f...,doa rezeki tak putus inna larizquna lahu min f...,"[doa, rezeki, putus, inna, larizquna, lahu, mi...",doa rezeki putus inna larizquna lahu min fades...
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,,makasih loh ntar kita bagi hasil aku sisanya b...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar kita bagi hasil aku sis...,"[terima, kasih, loh, entar, hasil, sisa]",terima kasih loh entar hasil sisa
4,-1,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,"[faham, jenis, orang, malaysia, ret, nak, sang...",faham jenis orang malaysia ret nak sangkut hos...


In [ ]:
df1.shape

(9772, 11)

In [ ]:
df2 = df1[df1['cleantext'].str.contains('mood booster') == False]
df2 = df2[df2['cleantext'].str.contains('pmk') == False]
df2 = df2[df2['cleantext'].str.contains('kucing') == False]
df2 = df2[df2['cleantext'].str.contains('rabies') == False]
df2 = df2[df2['cleantext'].str.contains('hewan') == False]
df2.shape

(6170, 12)

In [ ]:
df3=df1[df1['tokens'].map(lambda d: len(d)) > 1]
df3.shape

(9375, 11)

In [ ]:
df3['date'] = pd.to_datetime(df3['date'])
print(type(df3.date[0]))
df3.sort_values(by='date',ascending=True)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,user,text,casefold,extract_mentions,extract_hashtags,emoji,cleaned,normalized,stemmed,tokens,cleantext
3227,2022-04-05 22:24:00,NTSertrodimedjo,"Kl saya booster standard kl drop habbats, madu...","kl saya booster standard kl drop habbats, madu...","kl saya booster standard kl drop habbats, madu...","kl saya booster standard kl drop habbats, madu...",,saya booster standard drop habbats madu dan bi...,saya booster standard drop habbats madu dan bi...,saya booster standard drop habbats madu dan bi...,"[booster, standard, drop, habbats, madu, biang...",booster standard drop habbats madu biang wedan...
3225,2022-04-05 22:29:00,_Mood_Booster,@blinggdulce Sakit dikit kook.. ð,@blinggdulce sakit dikit kook.. ð,sakit dikit kook.. ð,sakit dikit kook.. ð,,sakit dikit kook,sakit dikit kook,sakit dikit kook,"[sakit, dikit, kook]",sakit dikit kook
3224,2022-04-05 22:42:00,darmajasa,RT @D_RyanWijaya: efek samping booster selain ...,rt @d_ryanwijaya: efek samping booster selain ...,"efek samping booster selain gampang laper, jdi...","efek samping booster selain gampang laper, jdi...",,efek samping booster selain gampang laper jdi ...,efek samping booster selain gampang laper jadi...,efek samping booster selain gampang laper jadi...,"[efek, samping, booster, gampang, laper, gampa...",efek samping booster gampang laper gampang ken...
3222,2022-04-05 22:47:00,BuKasun_Ndeso,"RT @GratisTerbaik: Yes! Semua PNS, karyawan da...","rt @gratisterbaik: yes! semua pns, karyawan da...","yes! semua pns, karyawan dan orang lain yang d...","yes! semua pns, karyawan dan orang lain yang d...",,yes semua pns karyawan dan orang lain yang dig...,yes semua pns karyawan dan orang lain yang dig...,yes semua pns karyawan dan orang lain yang gan...,"[yes, pns, karyawan, orang, ganggu, aparat, bo...",yes pns karyawan orang ganggu aparat booster t...
3221,2022-04-05 22:48:00,mudha_satriya,RT @gibran_tweet: Mendampingi Pak @sandiuno Me...,rt @gibran_tweet: mendampingi pak @sandiuno me...,mendampingi pak menparekraf ngecek kesiapan m...,mendampingi pak menparekraf ngecek kesiapan m...,,mendampingi pak menparekraf ngecek kesiapan mu...,mendampingi pak menparekraf mengecek kesiapan ...,damping pak menparekraf ecek kesiap mudik term...,"[damping, menparekraf, ecek, kesiap, mudik, te...",damping menparekraf ecek kesiap mudik terminal...
...,...,...,...,...,...,...,...,...,...,...,...,...
1205,2022-12-10 23:58:00,pakd_im2,RT @tamim_pardede: tdk bisa dipungkiri bahwa m...,rt @tamim_pardede: tdk bisa dipungkiri bahwa m...,tdk bisa dipungkiri bahwa materi dari virus pe...,tdk bisa dipungkiri bahwa materi dari virus pe...,,tdk bisa dipungkiri bahwa materi dari virus pe...,tidak bisa dipungkiri bahwa materi dari virus ...,tidak bisa dipungkiri bahwa materi dari virus ...,"[dipungkiri, materi, virus, hiv, aids, bahan, ...",dipungkiri materi virus hiv aids bahan baku va...
609,2022-12-10 23:58:00,catatankaqihati,RT @riea_mariska86: Vaksinasi sebagai upaya pe...,rt @riea_mariska86: vaksinasi sebagai upaya pe...,vaksinasi sebagai upaya pencegahan primer yang...,vaksinasi sebagai upaya pencegahan primer yang...,,vaksinasi sebagai upaya pencegahan primer yang...,vaksinasi sebagai upaya pencegahan primer yang...,vaksinasi bagai upaya cegah primer yang sangat...,"[vaksinasi, upaya, cegah, primer, handal, cega...",vaksinasi upaya cegah primer handal cegah saki...
1207,2022-12-10 23:58:00,catatankaqihati,"RT @garuda080: Vaksin Covid-19 aman, bermanfaa...","rt @garuda080: vaksin covid-19 aman, bermanfaa...","vaksin covid-19 aman, bermanfaat untuk membent...","vaksin covid-19 aman, bermanfaat untuk membent...",,vaksin covid aman bermanfat untuk membentuk ke...,vaksin covid aman bermanfat untuk membentuk ke...,vaksin covid aman bermanfat untuk bentuk kebal...,"[vaksin, covid, aman, bermanfat, bentuk, kebal...",vaksin covid aman bermanfat bentuk kebal tubuh...
608,2022-12-10 23:59:00,catatankaqihati,RT @Anggita_lung: Vaksinasi dan Prokes dua kun...,rt @anggita_lung: vaksinasi da

In [ ]:
df3.head()

,date,user,text,casefold,extract_mentions,extract_hashtags,emoji,cleaned,normalized,stemmed,tokens,cleantext
0,2022-10-23 23:57:00,UmarArradeny,Srimulyani sudah mengeluarkan pernyataan bahay...,srimulyani sudah mengeluarkan pernyataan bahay...,srimulyani sudah mengeluarkan pernyataan bahay...,srimulyani sudah mengeluarkan pernyataan bahay...,,srimulyani sudah mengeluarkan pernyatan bahaya...,srimulyani sudah mengeluarkan pernyatan bahaya...,srimulyani sudah keluar pernyatan bahaya moga ...,"[srimulyani, pernyatan, bahaya, moga, hidup, i...",srimulyani pernyatan bahaya moga hidup ilusi v...
1,2022-10-23 23:57:00,mbokewongndeso,RT @elizabethlisa76: Poly ethilen Glycol yg di...,rt @elizabethlisa76: poly ethilen glycol yg di...,poly ethilen glycol yg digadang ada dilm obat ...,poly ethilen glycol yg digadang ada dilm obat ...,,poly ethilen glycol digadang ada dilm obat bat...,poly ethilen glycol digadang ada dilm obat bat...,poly ethilen glycol gadang ada dilm obat batuk...,"[poly, ethilen, glycol, gadang, dilm, obat, ba...",poly ethilen glycol gadang dilm obat batuk sir...
2,2022-10-23 23:57:00,Rizvan168,"RT @HeyFarahhh: Lanjut yak.\n\nJadi, oleh pene...","rt @heyfarahhh: lanjut yak.\n\njadi, oleh pene...","lanjut yak.\n\njadi, oleh penelitian yg dilaku...","lanjut yak.\n\njadi, oleh penelitian yg dilaku...",,lanjut yak jadi oleh penelitian dilakukan drh ...,lanjut ya jadi oleh penelitian dilakukan drh i...,lanjut ya jadi oleh teliti laku drh indro ada ...,"[teliti, laku, drh, indro, kecurigan, kalo, ga...",teliti laku drh indro kecurigan kalo gagal gin...
4,2022-10-23 23:56:00,adzit7,"kata mereka yg antivak sih, ortunya yg vaksin ...","kata mereka yg antivak sih, ortunya yg vaksin ...","kata mereka yg antivak sih, ortunya yg vaksin ...","kata mereka yg antivak sih, ortunya yg vaksin ...",,kata mereka antivak sih ortunya vaksin trus va...,kata mereka antivak sih orang tuanya vaksin te...,kata mereka antivak sih orang tua vaksin terus...,"[antivak, sih, orang, tua, vaksin, vaksin, mas...",antivak sih orang tua vaksin vaksin masuk anak...
5,2022-10-23 23:56:00,sarkasimo,Doktif dokter fiktif ..\nMeyakini benar itulah...,doktif dokter fiktif ..\nmeyakini benar itulah...,doktif dokter fiktif ..\nmeyakini benar itulah...,doktif dokter fiktif ..\nmeyakini benar itulah...,,doktif dokter fiktif meyakini benar itulah pri...,doktif dokter fiktif meyakini benar itulah pri...,doktif dokter fiktif yakin benar itu prinsip w...,"[doktif, dokter, fiktif, prinsip, salah, nama,...",doktif dokter fiktif prinsip salah nama dokter...


In [ ]:
df3.to_csv('random preprocessed dataset.csv', sep=';', index=False, encoding= 'utf-8')

In [ ]:
df3.to_csv('preprocessed 6000.csv', sep=';', columns=['date','user','text','cleantext','tokens'], index=False, encoding= 'utf-8')

In [ ]:
from google.colab import files
files.download("random preprocessed dataset.csv")
#files.download("preprocessed 6000.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>